In [1]:
from bs4 import BeautifulSoup
import requests
# import pandas as pd
import time
import re
import urllib
import os

In [2]:
DATAFOLDER = '../data/irasutoya/'
print(os.listdir(DATAFOLDER))

[]


In [139]:
url = 'https://www.irasutoya.com/2020/01/blog-post_335.html'

In [123]:
def get_categories(url):
    soup = BeautifulSoup(requests.get(url).content)
    cat_url_list = []
    cat_name_list = []
    href_list = soup.find('div', {'class':'widget-content list-label-widget-content'}).find_all('a')
    for href in href_list:
        cat_url_list.append(href['href'])
        cat_name_list.append(href.text)
    return cat_url_list, cat_name_list

In [137]:
def get_image_urls(cat_url):
    soup = BeautifulSoup(requests.get(cat_url).content)
    img_url_list = []
    img_name_list = []
    img_href_list = soup.find_all('div', {'class':"boxmeta clearfix"})

    while(True):
        for href in img_href_list:
            img_url_list.append(href.find('a')['href'])
            img_name_list.append(href.find('a').text)
        next_url = soup.find('div', {'class':'blog-pager'}).find('a', {'title':"前の投稿"})['href']
        if next_url == None:
            break
        soup = BeautifulSoup(requests.get(next_url).content)
        img_href_list = soup.find_all('div', {'class':"boxmeta clearfix"})
    return img_url_list, img_name_list

KeyboardInterrupt: 

In [144]:
cat_url_list[0]

'https://www.irasutoya.com/search/label/%E3%81%93%E3%81%A9%E3%82%82'

In [140]:
def get_image(img_url):
    soup = BeautifulSoup(requests.get(img_url).content)
    images = soup.find_all('div', {'class':"separator"})
    for obj in images:
        obj_row = obj.find_all('a')
        for ob in obj_row:
            try:
                img_url = ob.find('img')['src']
                img_title = ob.find('img')['alt']
                img_name = img_url.split('/')[-1]
                print(img_title, img_name,  img_url)
            except:
                break
    caption = images[-1].text
    print(caption)

In [141]:
url = 'https://www.irasutoya.com/2020/01/blog-post_335.html'
get_image(url)

レントゲン写真を撮る子供のイラスト（男の子） medical_xray_rentogen_boy.png https://1.bp.blogspot.com/-ZsV56hrD2q0/Xexq31b0RPI/AAAAAAABWjg/9OxhAwE7kygAsDPrqbzIYeMbdM-Y35IZwCNcBGAsYHQ/s350/medical_xray_rentogen_boy.png
レントゲン写真を撮る子供のイラスト（女の子） medical_xray_rentogen_girl.png https://1.bp.blogspot.com/-1z9ptUcy76E/Xexq5-nb4hI/AAAAAAABWjw/cCXPAb2HM5QyxVM_7BoJ23fPkdn5n1InQCNcBGAsYHQ/s350/medical_xray_rentogen_girl.png
胸部のレントゲンを撮影している男の子と女の子のイラストです。


In [89]:
path = DATAFOLDER + img_name
data = urllib.request.urlopen(img_url).read()
with open(path, mode="wb") as f:
    f.write(data)

In [124]:
cat_url_list, cat_name_list = get_categories(url)
cat_url_list

['https://www.irasutoya.com/search/label/%E3%81%93%E3%81%A9%E3%82%82',
 'https://www.irasutoya.com/search/label/%E8%81%B7%E6%A5%AD',
 'https://www.irasutoya.com/search/label/%E7%97%85%E6%B0%97',
 'https://www.irasutoya.com/search/label/%E3%83%9D%E3%83%BC%E3%82%BA',
 'https://www.irasutoya.com/search/label/%E4%BC%9A%E7%A4%BE',
 'https://www.irasutoya.com/search/label/%E3%81%8A%E9%87%91',
 'https://www.irasutoya.com/search/label/%E9%81%93%E5%85%B7',
 'https://www.irasutoya.com/search/label/%E9%81%95%E5%8F%8D',
 'https://www.irasutoya.com/search/label/%E3%83%93%E3%82%B8%E3%83%8D%E3%82%B9',
 'https://www.irasutoya.com/search/label/%E5%AD%A6%E6%A0%A1',
 'https://www.irasutoya.com/search/label/%E3%83%95%E3%82%A1%E3%83%83%E3%82%B7%E3%83%A7%E3%83%B3',
 'https://www.irasutoya.com/search/label/%E4%BA%8B%E6%95%85',
 'https://www.irasutoya.com/search/label/%E9%A3%9F%E3%81%B9%E7%89%A9',
 'https://www.irasutoya.com/search/label/%E5%8C%BB%E7%99%82',
 'https://www.irasutoya.com/search/label/%E8%B6%A3%

In [ ]:
import sqlite3
conn = sqlite3.connect('irasutoya.db')
c = conn.cursor()
c.execute('create table irasuto(id integer, name text, caption text, discription text)')